In [1]:
import tensorflow as tf

In [2]:
raw_base_model = tf.keras.applications.EfficientNetV2S(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    include_preprocessing=True,
)


87549136/87549136 [==============================] - 2s 0us/step


In [3]:
raw_base_model.summary()

Model: "efficientnetv2-s"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384, 384, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 384, 384, 3)  0           ['input_1[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 192, 192, 24  648         ['rescaling[0][0]']              
                                )                                                                 
                                                                                   

In [37]:
cifar = tf.keras.datasets.cifar10.load_data()
(x_train, y_train), (x_test, y_test) = cifar

# y_train_ohe = tf.one_hot([i[0] for i in  y_train],10)
# indexes_for_rows = tf.range(0,y_train.shape[0])
# shuffled_indexes = tf.random.shuffle(indexes_for_rows)
# selected_indexes = shuffled_indexes[:TRAINING_SET_SIZE]
# selected_x_train = x_train[selected_indexes,:,:,:]
# selected_y_train_ohe = y_train_ohe.numpy()[selected_indexes,:]

170498071/170498071 [==============================] - 2s 0us/step


In [39]:
x_train[0].shape

(32, 32, 3)

In [43]:

def get_efficientnet_v2s_base_model():
    raw_base_model = tf.keras.applications.EfficientNetV2S(
        include_top=True,
        weights="imagenet",
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1000,
        classifier_activation="softmax",
        include_preprocessing=True)
    unflattened_base_model_output = raw_base_model.layers[-6].output
    flattened_base_model_output = tf.keras.layers.Flatten()(unflattened_base_model_output)
    initial_base_model = tf.keras.Model(inputs=raw_base_model.inputs,outputs=flattened_base_model_output)
    
    initial_input = tf.keras.layers.Input(shape=(32,32,3))
    resized =\
        tf.keras.layers.Resizing(
        384, 384, interpolation="bilinear")(initial_input)
    final_model_output = initial_base_model(resized)
    
    final_base_model = tf.keras.Model(inputs=initial_input,outputs=final_model_output)
    
    

    
    return final_base_model


In [44]:
final_efficientnet_v2s_base_model = get_efficientnet_v2s_base_model()

In [45]:
final_efficientnet_v2s_base_model.summary()




Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 resizing (Resizing)         (None, 384, 384, 3)       0         
                                                                 
 model_9 (Functional)        (None, 184320)            20326240  
                                                                 
Total params: 20,326,240
Trainable params: 20,174,928
Non-trainable params: 151,312
_________________________________________________________________


In [49]:
test_123 = final_efficientnet_v2s_base_model(x_train[:100])